## set the var env and the global var

In [68]:
%reset -f
import os
from dotenv import load_dotenv
from datetime import datetime

# Get the SEO keywords and parent folder as input from the user
seo_keywords = "amazigh maroc berbere" 

# 1: moroccoheritage 2 : gobitcode
website = "1"
current_date = datetime.now().strftime('%Y-%m-%d')
# app requirements


In [69]:
if website == '1':
    file_path = f'./format/moroccoheritage.mdx'
    blog_path = "/home/adil/repo/morocco-heritage/data/blog/"
    image_path = "/home/adil/repo/morocco-heritage/public/static/images"
else:
    file_path = f'./format/gobitcode.mdx'
    blog_path = "/home/adil/repo/gobitcode/data/blog/"
    image_path = "/home/adil/repo/gobitcode/public/static/images"

with open(file_path, 'r') as file:
    mdx_format = file.read()


mdx_format = mdx_format.replace("current_date", current_date)

# Load environment variables from .env file
load_dotenv()

# Get the API key from the environment variable
groq_api_key = os.getenv('GROQ_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
google_cx = os.getenv('GOOGLE_CX')

print(mdx_format)


---
title: The Timeless Elegance of the Moroccan Djellaba for Men
date: '2025-03-02'  
tags: ['moroccan fashion', 'traditional clothing', 'djellaba']
draft: false
summary: Explore the cultural significance and enduring appeal of the Moroccan Djellaba for men, a garment that represents both tradition and style in Morocco.
---

## Introduction

Brief introduction about the **Moroccan Djellaba** and its cultural importance.

<TOCInline toc={props.toc} exclude="Introduction" />

## What is the Moroccan Djellaba?

Explanation of the **djellaba**, its materials, and uses.

<img src="/static/images/image1.png" alt="Moroccan Djellaba" width="400" height="300" />

## A Brief History of the Djellaba

Summary of the **djellaba**'s origins and evolution over time.

## Types of Moroccan Djellabas for Men

- **Winter Djellaba**
- **Summer Djellaba**
- **Ceremonial Djellaba**

<img src="/static/images/image2.png" alt="Modern Moroccan Djellaba" width="500" height="300" />

## Craftsmanship Behind the 

## ai generate the blog

In [70]:
from groq import Groq

client = Groq(api_key=groq_api_key)

completion = client.chat.completions.create(
    model="llama3-70b-8192",
    messages=[
        {
            "role": "system",
            "content": f"You are a blog writer who writes blogs in this format: \n {mdx_format} \n Do not include any introductory text."
        },
        {
            "role": "user",
            "content": f"Write a long-form blog about '{seo_keywords}' \n Do not forget image links and alt text."
        }
    ],
    temperature=1,
    max_tokens=1024*4,
    top_p=1,
    stream=False,  # Set stream to False to get the response as one
    stop=None,
)

mdx_blog = completion.choices[0].message.content

print(mdx_blog)

---
title: Unveiling the Rich Heritage of Amazigh Maroc Berbere: Preserving the Indigenous Identity of Morocco
date: '2025-03-02'
tags: ['amazigh culture', 'berber heritage', 'moroccan identity']
draft: false
summary: Delve into the fascinating world of Amazigh Maroc Berbere, exploring their history, language, customs, and struggle for recognition in modern Morocco.
---

## Introduction

In the midst of Morocco's vibrant cultural landscape, there exists a rich and resilient heritage that has been waiting to be unveiled. The Amazigh Maroc Berbere, also known as the Berbers or Imazighen, are the indigenous people of Morocco, with a history that predates the Arab conquest of the region. Despite their significant contribution to Moroccan identity, the Amazigh people have long been marginalized, their language and culture threatened by the dominant Arab influence. In this blog, we embark on a journey to explore the fascinating world of Amazigh Maroc Berbere, their history, language, customs

## slugify

In [71]:
import re
# Function to generate a slug from the title
def generate_slug(title):
    # Convert to lowercase
    slug = title.lower()
    # Replace spaces and special characters with hyphens
    slug = re.sub(r'[^a-z0-9]+', '-', slug)
    # Remove leading and trailing hyphens
    slug = slug.strip('-')
    return slug


# Regular expression to extract the title
title_match = re.search(r'^title:\s*(.*)$', mdx_blog, re.MULTILINE)

if title_match:
    title = title_match.group(1)
    slug = generate_slug(title)
else:
    raise ValueError("Title not found in the blog")

parent_folder = generate_slug(seo_keywords)

print(f"Slug: {slug}")

Slug: unveiling-the-rich-heritage-of-amazigh-maroc-berbere-preserving-the-indigenous-identity-of-morocco


## download images

In [72]:
from google_images_search import GoogleImagesSearch

def download_images(keyword,path,  limit=1):
    # You need to get your own API key and CX from Google Custom Search JSON API
    gis = GoogleImagesSearch(google_api_key,google_cx)

    search_params = {
        'q': keyword,
        'num': limit,
        'safe': 'off',
        'fileType': 'jpg|png',
        'imgType': 'photo',
        'imgSize': 'LARGE',  # Use a single size or remove this line if not needed
        # 'imgDominantColor': 'black'  # Use a single color or remove this line if not needed
    }

    gis.search(search_params=search_params)
    image = gis.results()[0]
    image_save_path = os.path.join(image_path,path)
    image.download(image_save_path)
    image_name = os.path.basename(image.path)
    return image_name




In [73]:



# Regex to match the <img> tag and capture the alt attribute
img_tag_regex = r'<img[^>]*alt="([^"]*)"[^>]*>'

# Find all matches
matches = re.findall(img_tag_regex, mdx_blog)



# Print the alt attributes
for match in matches:
    image_name =  download_images(f"{seo_keywords} {match}"  , parent_folder)
    print(f"alt attribute: {match}")
    new_src = f'/static/images/{parent_folder}/{image_name}'
    mdx_blog = re.sub(rf'(<img[^>]*src=")[^"]*("[^>]*alt="{re.escape(match)}"[^>]*>)', rf'\1{new_src}\2', mdx_blog)
    
print(mdx_blog) 
    

alt attribute: Ancient Amazigh History
alt attribute: Tamazight Script
alt attribute: Imilchil Marriage Festival
alt attribute: Amazigh Protest
alt attribute: Amazigh Textiles
---
title: Unveiling the Rich Heritage of Amazigh Maroc Berbere: Preserving the Indigenous Identity of Morocco
date: '2025-03-02'
tags: ['amazigh culture', 'berber heritage', 'moroccan identity']
draft: false
summary: Delve into the fascinating world of Amazigh Maroc Berbere, exploring their history, language, customs, and struggle for recognition in modern Morocco.
---

## Introduction

In the midst of Morocco's vibrant cultural landscape, there exists a rich and resilient heritage that has been waiting to be unveiled. The Amazigh Maroc Berbere, also known as the Berbers or Imazighen, are the indigenous people of Morocco, with a history that predates the Arab conquest of the region. Despite their significant contribution to Moroccan identity, the Amazigh people have long been marginalized, their language and cul

## correct title

In [74]:



# Regex to match and remove the colon from the title line
title_regex = r'^(title:.*?):(.*)$'

# Remove the colon from the title line
mdx_blog = re.sub(title_regex, r'\1\2', mdx_blog, flags=re.MULTILINE)

print(mdx_blog)

---
title: Unveiling the Rich Heritage of Amazigh Maroc Berbere Preserving the Indigenous Identity of Morocco
date: '2025-03-02'
tags: ['amazigh culture', 'berber heritage', 'moroccan identity']
draft: false
summary: Delve into the fascinating world of Amazigh Maroc Berbere, exploring their history, language, customs, and struggle for recognition in modern Morocco.
---

## Introduction

In the midst of Morocco's vibrant cultural landscape, there exists a rich and resilient heritage that has been waiting to be unveiled. The Amazigh Maroc Berbere, also known as the Berbers or Imazighen, are the indigenous people of Morocco, with a history that predates the Arab conquest of the region. Despite their significant contribution to Moroccan identity, the Amazigh people have long been marginalized, their language and culture threatened by the dominant Arab influence. In this blog, we embark on a journey to explore the fascinating world of Amazigh Maroc Berbere, their history, language, customs,

## save the blog

In [75]:
filename = f"{slug}.mdx"

blog_save_path = os.path.join(blog_path, filename)

with open(blog_save_path, 'w') as file:
    file.write(mdx_blog)

print(f"File '{filename}' created successfully.")

File 'unveiling-the-rich-heritage-of-amazigh-maroc-berbere-preserving-the-indigenous-identity-of-morocco.mdx' created successfully.
